In [2]:
from model import atp_graph, losses
from data import synthetic_data_gen, feature_extractor
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from model import atp_pipeline
from data import dataset_preparer
import argparse
from Tutorials.helper import batcher
import os

In [3]:
dataset = "weather"
model = "atp"
iterations = 300
num_repeats = 1
n_C = 20
n_T = 40

In [4]:


if dataset == "weather":
    x_train, y_train, x_val, y_val, x_test, y_test = dataset_preparer.weather_processor(path_to_weather_data="datasets/weather.csv") 
    save_dir = "weights/forecasting/weather"


batch_size = 32
test_batch_s = 100

nll_list = []
mse_list = []

for i in range(num_repeats):

    step = 1
    run= 50 + i
    tf.random.set_seed(run)

    if model == "atp":
        model = atp_pipeline.instantiate_atp(dataset)

    if model == "tnp":
        model = tnp_pipeline.instantiate_tnp(dataset)

    tr_step = atp_graph.build_graph()

    ###### can we put the name of the model into the folder name #########?

    name_comp = 'run_' + str(run)
    folder = save_dir + '/ckpt/check_' + name_comp
    if not os.path.exists(folder): os.mkdir(folder)
    opt = tf.keras.optimizers.Adam(3e-4)
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=model)
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
    ckpt.restore(manager.latest_checkpoint) 
    sum_mse_tot = 0; sum_nll_tot = 0
    mini = 50000


    for i in range(iterations):
        idx_list = list(range(x_train.shape[0] - (n_C+n_T)))
        x,y,_ = batcher(x_train,y_train,idx_list,window=n_C+n_T) ####### generalise for not just forecasting
        _,_, _, _ = tr_step(model, opt, x,y,n_C,n_T, training=True)

        if i % 100 == 0:
            print(i)
            idx_list = list(range(x_val.shape[0] - (n_C+n_T)))
            t_te,y_te,_ = batcher(x_val,y_val,idx_list,batch_s = 100,window=n_C+n_T)
            μ, log_σ = model([t_te, y_te, n_C, n_T, False])
            _,_,_, nll_pp_te, msex_te = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)

            if nll_pp_te < mini:
                mini = nll_pp_te
                manager.save()
                step += 1
                ckpt.step.assign_add(1)

    ckpt = tf.train.Checkpoint(step=tf.Variable(step), optimizer=opt, net=model)
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
    ckpt.restore(manager.latest_checkpoint) 
    #### test the restore function ####

#   




Metal device set to: Apple M1 Max


2023-05-03 17:25:43.439799: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-03 17:25:43.439938: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Tensor("atp_pipeline/argsort/TopKV2:1", shape=(1, 40), dtype=int32)
Tensor("Const_2:0", shape=(), dtype=bool)
Tensor("atp_pipeline/argsort/TopKV2:1", shape=(1, 40), dtype=int32)
Tensor("Const_2:0", shape=(), dtype=bool)


2023-05-03 17:25:46.437674: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-03 17:25:46.438238: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


0
tf.Tensor(
[[26 10 28  9 39  1 12 31 17 34  3 21 37 38 13 25 35 27 18  6  5 14  2 32
  20 15 22 29 23 16  7 24 36 30 11  8  4 19 33  0]], shape=(1, 40), dtype=int32)
tf.Tensor(False, shape=(), dtype=bool)
100
tf.Tensor(
[[27 39 15 35 12 22  9  0 31 18  5 23 32 37 16 29 25  2 26 38 17  1 36  7
  24  6 11 10 19 28 20  3  8 33  4 21 14 30 13 34]], shape=(1, 40), dtype=int32)
tf.Tensor(False, shape=(), dtype=bool)
200
tf.Tensor(
[[28 38 20 33  6 39  9  8  2 27 14 16 18 15 35 31  3 34 32 37  5  4 23 17
  29 25 19 10 24 22 30 26 12  7 36 11 13  1 21  0]], shape=(1, 40), dtype=int32)
tf.Tensor(False, shape=(), dtype=bool)


In [5]:
x_test.shape[0]

10540

In [8]:
105 * 100

10500

In [9]:
x_test.shape[0]//test_batch_s

105

In [12]:
(n_C+n_T)

60

In [14]:
test_batch_s = 100
idx_list = list(range(x_test.shape[0] - (n_C+n_T)))
num_batches = x_test.shape[0]//test_batch_s
for _ in range(num_batches): #### specify correct number of batches for the batcher #####
    if(_ == (num_batches-1)): test_batch_s = len(idx_list)
    print(len(idx_list))
    print(test_batch_s)
    t_te,y_te,idx_list = batcher(x_test, y_test, idx_list,batch_s = test_batch_s, window=n_C+n_T)
    μ, log_σ = model([t_te, y_te, n_C, n_T, False])
    _, sum_mse, sum_nll, _, _ = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)
    sum_nll_tot += sum_nll / n_T
    sum_mse_tot += sum_mse / n_T

nllx =  sum_nll_tot / (test_batch_s * x_test.shape[0]//test_batch_s)
msex =  sum_mse_tot / (test_batch_s * x_test.shape[0]//test_batch_s)


nll_list.append(nllx.numpy())
mse_list.append(msex.numpy())

#     np.save(save_dir + '/nll_list.npy', nll_list)    
#     np.save(save_dir + '/mse_list.npy', mse_list)  

10480
100
tf.Tensor(
[[11  5 24  4 27 22 30 16 34 31 37 12 28 32 13  1 29 19  0 36 26 38 14 10
   7  8  3 18 23 25  9 21  6 15 33 39 35 20 17  2]], shape=(1, 40), dtype=int32)
tf.Tensor(False, shape=(), dtype=bool)
10380
100
tf.Tensor(
[[25 37 20 30 38 23 35 36  6 12 26 21  1  0 17 31 29  4  2 34 32 24 22 11
  27 16 39 19  8 28  7  3  9 14 15 33  5 13 10 18]], shape=(1, 40), dtype=int32)
tf.Tensor(False, shape=(), dtype=bool)
10280
100
tf.Tensor(
[[21 19  8  7  6 37 29 36 34 35 33  2 10 14 25 22 18 28 24  9  0 17 11 32
  16 30 27 26 39 23 20 38  1 15 12  4  3  5 13 31]], shape=(1, 40), dtype=int32)
tf.Tensor(False, shape=(), dtype=bool)
10180
100
tf.Tensor(
[[ 6 32 30 18 27 16  7 37  9 29  0 12  3 39 25  5 15  8  4 22 33 36 23 19
   2 10 34 31 11 14 17 20 24  1 35 38 26 28 13 21]], shape=(1, 40), dtype=int32)
tf.Tensor(False, shape=(), dtype=bool)
10080
100
tf.Tensor(
[[30 29 37 15 10 27 19  1 12 18 25 20 34 33 38 32 21 36  0 14  7 28 16 13
   3 23 22  2 39  9  4 35  5 24 31  8 17 26  